# Preprocessing in Remote Sensing: Stacking

---

This section describes the fundamental preprocessing step of band stacking, which is essential for creating multispectral or hyperspectral image composites from individual spectral bands:

Band stacking is the process of combining multiple single-band raster images—each representing reflectance in a specific wavelength—into a single multi-band file. This is a crucial step for most remote sensing analyses, as many indices (e.g., NDVI, NDWI) or classification methods rely on the spectral relationship between bands.

Band stacking requires that all input images are perfectly coregistered and have the same spatial resolution, projection, and extent. If not, misalignment artifacts may appear and compromise the analysis.

---

## Import

In [1]:
from osgeo import gdal,osr,gdalconst
gdal.UseExceptions()

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
import os
import tempfile

---

## Optionnal

Band stacking can consumme a lot of computer ressource. If you are not sure of your computer capacity, you can bridle your CPU and memory to 90% effectivness.

In [2]:
import multiprocessing
import psutil

total_ram_bytes = psutil.virtual_memory().total #obtain your memory capacity
total_ram_mb = total_ram_bytes // (1024 * 1024) #convertion byte -> mb

cpu_count = multiprocessing.cpu_count() #obtain your cpu number

# 90% effectivness
ram_limit_mb = int(total_ram_mb * 0.9)
cpu_limit = max(1, int(cpu_count * 0.9))


gdal.SetCacheMax(ram_limit_mb * 1024 * 1024)  # We can set the cache capacity of gdal

print(f"Limitation GDAL :")
print(f" - RAM : {ram_limit_mb} Mo on {total_ram_mb} Mo")
print(f" - CPU : {cpu_limit} hearts on {cpu_count}")

Limitation GDAL :
 - RAM : 12769 Mo on 14188 Mo
 - CPU : 14 hearts on 16


---

## Band Stacking

In this example, we use Sentinel-2 data from corpernicus. We download 2 files : 
- S2A_MSIL2A_20190305T222531_N0500_R029_T59GPM_20221119T040431.SAFE.zip
- S2B_MSIL2A_20190119T222539_N0500_R029_T59GPM_20221217T001329.SAFE.zip

Next, we create a function who return the path of each band asked. We want B01,B02,B03,B04,B05,B06,B07,B8A,B11 and B12 at 20m resolution. This function have the particularity to create the correct path with zip file or normal file. This function work only for Sentinel-2 data.

In [3]:
def path_finder(data_path, bands, resolution):
    """
    Finds all GDAL-compatible paths (/vsizip/) to the requested tapes in a folder
    containing .SAFE or .SAFE.zip (Sentinel-2) files.

    Returns
    -------
    paths       : list[str]   path /vsizip/... to the band .jp2
    band_names  : list[str]   simple band name
    """
    resolution_dir = f'R{resolution}'
    paths, band_names = [], []

    # Create a list from each file within the main path
    files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.SAFE.zip') or f.endswith('.SAFE')]

    for item in files:
        # ZIP case
        if item.endswith('.zip'):
            abs_zip = os.path.abspath(item).replace("\\", "/")
            with zipfile.ZipFile(item, 'r') as z:
                jp2_inside = [
                    f for f in z.namelist()
                    if f.endswith(f"_{resolution}.jp2")
                    and f"/IMG_DATA/{resolution_dir}/" in f
                    and any(f"_B{b.replace('B','')}_{resolution}.jp2" in f for b in bands)
                ]
            for rel in jp2_inside:
                vsipath = f"/vsizip/{abs_zip}/{rel}"
                paths.append(vsipath)
                band_names.append(os.path.splitext(os.path.basename(rel))[0])

        # UNZIP case
        else:
            granule_dir = os.path.join(item, "GRANULE")
            granule_sub = next((os.path.join(granule_dir, d) for d in os.listdir(granule_dir)), None)
            img_dir = os.path.join(granule_sub, "IMG_DATA", resolution_dir)
            jp2_files = [
                f for f in os.listdir(img_dir)
                if f.endswith(f"_{resolution}.jp2")
                and any(f"_B{b.replace('B','')}_{resolution}.jp2" in f for b in bands)
            ]
            for f in jp2_files:
                fpath = os.path.join(img_dir, f).replace("\\", "/")
                paths.append(fpath)
                band_names.append(os.path.splitext(f)[0])


    return paths, band_names

path,desc=path_finder("data_copernicus_2",bands=['B01','B02','B03','B04','B05','B06','B07','B8A','B11','B12'],resolution='20m')

When we have a list of file, we can use gdal.buildVRT to create .vrt file. This step is used to prepare the input data for gdal.translate. In the options of gdal.buildVRT, we choose the parameters of the first image in the list as a reference and in order to make a stack we must choose "separate=True". Finally this vrt file is converted in geotif thank to gdal.translate. 

If you need, you can choose the parameter of gdal.translate. It's important to choose the right interleave,compression and tile paramaters.

In [5]:
liste_paths = path
liste_bande_name = desc

# First image = ref
ref_ds = gdal.Open(liste_paths[0])
gt = ref_ds.GetGeoTransform()
xres, yres = gt[1], abs(gt[5])
xmin = gt[0]
ymax = gt[3]
xmax = xmin + ref_ds.RasterXSize * xres
ymin = ymax - ref_ds.RasterYSize * yres
projection = ref_ds.GetProjection()
ref_ds = None

# Creation of temporary vrt file
with tempfile.NamedTemporaryFile(suffix=".vrt", delete=False) as tmp_vrt:
    vrt_path = tmp_vrt.name

vrt_options = gdal.BuildVRTOptions(
    resolution='user',
    xRes=xres,
    yRes=yres,
    outputBounds=[xmin, ymin, xmax, ymax],
    outputSRS=projection,
    resampleAlg="bilinear",
    separate=True #Important for stacking
)
vrt_ds = gdal.BuildVRT(
    "",
    srcDSOrSrcDSTab=liste_paths,
    options=vrt_options
)

# VRT to geotif
ds=gdal.Translate(
    "stacked.tif",
    vrt_ds,
    creationOptions=["COMPRESS=LZW", "TILED=YES", "INTERLEAVE=BAND","BIGTIFF=yes",f"NUM_THREADS={cpu_limit}"]
)
ds=None
vrt_ds = None
os.remove(vrt_path) #remove the vrt file

# If we want band name
ds_out = gdal.Open("stacked.tif", gdal.GA_Update)
for i, band_name in enumerate(liste_bande_name, start=1):
    band = ds_out.GetRasterBand(i)
    band.SetDescription(band_name)
ds_out.FlushCache()
ds_out = None